# Data preprocessing and cleaning

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


warnings.filterwarnings("ignore")
sns.set_style("whitegrid")

In [2]:
from src.config import (HOUSE_RAW_FILE, HOUSE_RAW_EXCEL_FILE, HOUSE_CLEAN_FILE,
                        INCOME_RAW_FILE, INCOME_CLEAN_TOTAL, INCOME_CLEAN_EDU,
                        AGE_RAW_FILE, AGE_CLEAN_FILE,
                        DENSITY_RAW_FILE, DENSITY_CLEAN_FILE,
                        WEATHER_FILE, WEATHER_QUARTER_FILE,
                        SERVICES_FILE, SERVICES_COLUMNS, MASTER_DF_FILE,
                        COLUMN_MISSING_VALUES, MUNICIPALITIES_FILE)

In [3]:
NUTS4 = pd.read_excel(HOUSE_RAW_EXCEL_FILE, sheet_name="Municipality", skiprows=3, header=None, skipfooter=2)

indexes_length_1 = NUTS4[0][NUTS4[0].str.len() == 1].index.tolist()
indexes_length_2 = NUTS4[0][NUTS4[0].str.len() == 2].index.tolist()
indexes_length_3 = NUTS4[0][NUTS4[0].str.len() == 3].index.tolist()

NUTS1 = NUTS4.iloc[indexes_length_1][1]
NUTS2 = NUTS4.iloc[indexes_length_2][1]
NUTS3 = NUTS4.iloc[indexes_length_3][1]

mun_to_drop = pd.concat([NUTS1, NUTS2, NUTS3]).unique()

MUNICIPALITIES = NUTS4[~NUTS4.isin(mun_to_drop)].dropna(ignore_index=True)[1]
NUTS4 = NUTS4[1]

## House pricing dataset

Data source: [PORDATA]()
The file contained 3 distinct groups of columns. We need only the average price per quarter, those are the 25-46 columns.

In [11]:
pd.read_csv(HOUSE_RAW_FILE, encoding="latin-1", sep=";")

,ï¿½ndice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unidade: ï¿½.2,Unnamed: 70
0,COD (NUTS2024),Designaï¿½ï¿½o,1ï¿½ Quartil das vendas por mï¿½ de alojamento...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1ï¿½T2025,4ï¿½T 2024,3ï¿½T2024,2ï¿½T2024,1ï¿½T2024,4ï¿½T 2023,3ï¿½T2023,2ï¿½T2023,...,4ï¿½T 2021,3ï¿½T 2021,2ï¿½T 2021,1ï¿½T 2021,4ï¿½T 2020,3ï¿½T 2020,2ï¿½T 2020,1ï¿½T 2020,4ï¿½T 2019,NaN
2,PT,Portugal,1 066,1 008,972,948,939,938,923,912,...,1 924,1 853,1 801,1 763,1 754,1 724,1 699,1 681,1 639,NaN
3,1,Continente,1 069,1 013,975,951,942,941,926,915,...,1 934,1 864,1 812,1 771,1 763,1 733,1 708,1 692,1 650,NaN
4,11,Norte,1 011,962,921,902,892,889,878,864,...,1 574,1 525,1 488,1 458,1 423,1 382,1 351,1 329,1 289,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,3003110,Sï¿½o Vicente,558,615,637,583,533,479,538,587,...,1 299,1 230,1 134,//,1 148,1 259,1 266,1 258,//,NaN
349,"Fonte: INE, I.P., Estatisticas de Preï¿½os da ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Last 4 rows can be considered as footer and first 3 are a corrupted header, so we'll skip those.

In [5]:
house_raw_df = pd.read_csv(HOUSE_RAW_FILE, 
                           encoding="latin-1", sep=";",
                           header=None, skiprows=3, skipfooter=4)
house_raw_df.head()

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,PT,Portugal,1 066,1 008,972,948,939,938,923,912,...,1 924,1 853,1 801,1 763,1 754,1 724,1 699,1 681,1 639,NaN
1,1,Continente,1 069,1 013,975,951,942,941,926,915,...,1 934,1 864,1 812,1 771,1 763,1 733,1 708,1 692,1 650,NaN
2,11,Norte,1 011,962,921,902,892,889,878,864,...,1 574,1 525,1 488,1 458,1 423,1 382,1 351,1 329,1 289,NaN
3,111,Alto Minho,744,698,690,677,660,654,629,632,...,1 288,1 250,1 205,1 200,1 183,1 162,1 151,1 125,1 113,NaN
4,1111601,Arcos de Valdevez,417,401,404,417,397,408,390,380,...,1 012,1 048,1 000,1 035,1 081,1 035,1 029,975,961,NaN


In [10]:
house_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 71 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       347 non-null    object 
 1   1       347 non-null    object 
 2   2       347 non-null    object 
 3   3       347 non-null    object 
 4   4       347 non-null    object 
 5   5       347 non-null    object 
 6   6       347 non-null    object 
 7   7       347 non-null    object 
 8   8       347 non-null    object 
 9   9       347 non-null    object 
 10  10      347 non-null    object 
 11  11      347 non-null    object 
 12  12      347 non-null    object 
 13  13      347 non-null    object 
 14  14      347 non-null    object 
 15  15      347 non-null    object 
 16  16      347 non-null    object 
 17  17      347 non-null    object 
 18  18      347 non-null    object 
 19  19      347 non-null    object 
 20  20      347 non-null    object 
 21  21      347 non-null    object 
 22  22

The initial csv file contains the entirely empty columns 24, 47 and 70. Those are the separation columns between 3 sections of the table: 
- 1st Quartile of dwellings sales per m² (in the last 12 months)[2-23]
- Median value per m² of dwellings sales (in the last 12 months)[25-46]
- 3rd Quartile of dwellings sales per m² (in the last 12 months)[48-69]

We'll use the 2nd column (Designation) and 26-47 columns (Median value per m² of dwellings sales).

The missing values are represented as "//" in the dataset, so we'll also deal with it. We'll drop the rows that have more than a half of columns with a missing value.

In [6]:
# Create new header
quarter_headers = []
years = range(2019, 2026)

for year in years:
    for quarter in range(1, 5):
        if year == 2025 and quarter != 1:
            continue
        if year == 2019 and quarter != 4:
            continue
        quarter_headers.append(f"{quarter}Q {year}")

quarter_headers.reverse()
house_header = ["Designation"] + quarter_headers

# Extract median prices
house_medians = house_raw_df.iloc[:, 25:47]

# Create a new DataFrame
house_df = pd.concat([NUTS4, house_medians], axis=1)
house_df.columns = house_header
house_df = house_df[house_df["Designation"].isin(MUNICIPALITIES)]
house_df.set_index("Designation", inplace=True)

house_df[COLUMN_MISSING_VALUES] = (house_df[quarter_headers] == '//').sum(axis=1)
house_df[COLUMN_MISSING_VALUES][house_df[COLUMN_MISSING_VALUES] > 0]

Designation
Boticas                      9
Ribeira de Pena              2
Carrazeda de Ansiães         1
Freixo de Espada à Cinta    15
Mesão Frio                  10
Murça                        8
Penedono                    22
Santa Marta de Penaguião     5
Sernancelhe                  1
Tabuaço                      7
Alfândega da Fé              2
Miranda do Douro             5
Vila Flor                    2
Vimioso                     14
Pampilhosa da Serra          2
Castanheira de Pêra          3
Aguiar da Beira              8
Vila Nova de Paiva          10
Oleiros                      1
Fornos de Algodres           6
Manteigas                    8
Alvito                       9
Barrancos                   21
Crato                        2
Fronteira                    1
Monforte                     1
Mourão                       1
Alcoutim                     5
Calheta                      8
Corvo                       22
Lajes das Flores            16
Santa Cruz da Graciosa     

In [7]:
for col in quarter_headers:
    tmp_series = pd.Series(house_df[col], dtype="str")
    tmp_series = tmp_series.replace(" ", "", regex=True)
    tmp_series = tmp_series.replace("//", np.nan, regex=False)
    house_df[col] = tmp_series.astype(float, errors="ignore")

In [8]:
house_df = house_df.dropna(thresh=10)
house_df = house_df.fillna(house_df.mean())
house_df = house_df.drop(columns=[COLUMN_MISSING_VALUES])
house_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 301 entries, Arcos de Valdevez to São Vicente
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   1Q 2025  301 non-null    float64
 1   4Q 2024  301 non-null    float64
 2   3Q 2024  301 non-null    float64
 3   2Q 2024  301 non-null    float64
 4   1Q 2024  301 non-null    float64
 5   4Q 2023  301 non-null    float64
 6   3Q 2023  301 non-null    float64
 7   2Q 2023  301 non-null    float64
 8   1Q 2023  301 non-null    float64
 9   4Q 2022  301 non-null    float64
 10  3Q 2022  301 non-null    float64
 11  2Q 2022  301 non-null    float64
 12  1Q 2022  301 non-null    float64
 13  4Q 2021  301 non-null    float64
 14  3Q 2021  301 non-null    float64
 15  2Q 2021  301 non-null    float64
 16  1Q 2021  301 non-null    float64
 17  4Q 2020  301 non-null    float64
 18  3Q 2020  301 non-null    float64
 19  2Q 2020  301 non-null    float64
 20  1Q 2020  301 non-null    float64
 2

In [9]:
# Save the municipalities list
MUNICIPALITIES = pd.Series(house_df.index.to_list())
MUNICIPALITIES.to_csv(MUNICIPALITIES_FILE, header=['region'])

house_df.to_csv(HOUSE_CLEAN_FILE)

In [15]:
house_df.head()

,1Q 2025,4Q 2024,3Q 2024,2Q 2024,1Q 2024,4Q 2023,3Q 2023,2Q 2023,1Q 2023,4Q 2022,...,1Q 2022,4Q 2021,3Q 2021,2Q 2021,1Q 2021,4Q 2020,3Q 2020,2Q 2020,1Q 2020,4Q 2019
Designation,,,,,,,,,,,,,,,,,,,,,
Arcos de Valdevez,960.0,958.0,1000.0,998.0,828.0,813.0,766.0,754.0,760.0,778.0,...,690.0,635.0,680.0,705.0,740.0,794.0,711.0,726.0,693.0,672.0
Caminha,1461.0,1425.0,1415.0,1334.0,1364.0,1318.0,1298.0,1280.0,1201.0,1201.0,...,1158.0,1111.0,1071.0,1000.0,1000.0,978.0,964.0,938.0,946.0,967.0
Melgaço,475.0,400.0,422.0,422.0,388.0,404.0,420.0,400.0,428.0,359.0,...,400.0,362.0,381.0,332.0,337.0,426.0,399.0,515.0,554.0,545.0
Monção,1244.0,1205.0,1244.0,1081.0,918.0,910.0,843.0,895.0,936.0,967.0,...,794.0,755.0,773.0,779.0,810.0,794.0,672.0,655.0,606.0,658.0
Paredes de Coura,770.0,708.0,743.0,722.0,753.0,723.0,600.0,645.0,565.0,587.0,...,654.0,652.0,594.0,531.0,511.0,509.0,479.0,479.0,500.0,513.0


## Income Dataset

Datasource: [PORDATA](https://www.pordata.pt/pt/estatisticas/economia/rendimento-e-poupanca/rendimento-disponivel-bruto-das-familias)

In [16]:
income_raw_df = pd.read_csv(INCOME_RAW_FILE, encoding="utf-8")
income_raw_df.head()

,01. Ano,02. Nome Região (Portugal),03. Âmbito Geográfico,04. Filtro 1,05. Filtro 2,06. Filtro 3,08. Símbolo,07. Escala,09. Valor
0,2013.0,Portugal,NaN,01. Inferior ao 1.º ciclo,NaN,NaN,x,euros,NaN
1,2013.0,Portugal,NaN,02. Básico - 1º Ciclo,NaN,NaN,x,euros,NaN
2,2013.0,Portugal,NaN,03. Básico - 2º Ciclo,NaN,NaN,x,euros,NaN
3,2013.0,Portugal,NaN,04. Básico - 3º Ciclo,NaN,NaN,x,euros,NaN
4,2013.0,Portugal,NaN,05. Secundário,NaN,NaN,x,euros,NaN


Map the portuguese column names to the english equivalents, leaving only the relevant for us features.

In [17]:
income_columns = {
    '01. Ano': 'year',
    '02. Nome Região (Portugal)': 'region',
    '03. Âmbito Geográfico': 'scope',
    '04. Filtro 1': 'education_level',
    '08. Símbolo': 'symbol',
    '07. Escala': 'unit',
    '09. Valor': 'value'
 }

In [20]:
income_df = income_raw_df.drop(columns=["05. Filtro 2", "06. Filtro 3"]).rename(columns=income_columns)
income_df = income_df.dropna(subset=["value", "year", "scope", "education_level"])
income_df["year"] = income_df["year"].astype(int)
income_df = income_df[income_df["scope"] == "Município"]
income_df = income_df[income_df["year"] >= 2019]
income_df

,year,region,scope,education_level,symbol,unit,value
20449,2019,Abrantes,Município,01. Inferior ao 1.º ciclo,NaN,euros,750.4
20450,2019,Abrantes,Município,02. Básico - 1º Ciclo,NaN,euros,939.7
20451,2019,Abrantes,Município,03. Básico - 2º Ciclo,NaN,euros,928.5
20452,2019,Abrantes,Município,04. Básico - 3º Ciclo,NaN,euros,933.5
20453,2019,Abrantes,Município,05. Secundário,NaN,euros,1027.3
...,...,...,...,...,...,...,...
37382,2023,Vouzela,Município,05. Secundário,NaN,euros,1082.9
37384,2023,Vouzela,Município,07. Bacharelato,NaN,euros,1357.6
37385,2023,Vouzela,Município,08. Licenciatura,NaN,euros,1562.9
37386,2023,Vouzela,Município,09. Mestrado,NaN,euros,1611.7


In [22]:
total_income_df = income_df[income_df['education_level'] == 'Total'].copy()
total_income_df.to_csv(INCOME_CLEAN_TOTAL, index=False)

total_income_df.head()

,year,region,scope,education_level,symbol,unit,value
20459,2019,Abrantes,Município,Total,NaN,euros,1109.4
20470,2019,Águeda,Município,Total,NaN,euros,1071.4
20481,2019,Aguiar da Beira,Município,Total,NaN,euros,818.6
20492,2019,Alandroal,Município,Total,NaN,euros,904.2
20503,2019,Albergaria-a-Velha,Município,Total,NaN,euros,1126.7


In [23]:
education_income_df = income_df[income_df['education_level'] != 'Total'].copy()
education_income_df.to_csv(INCOME_CLEAN_EDU, index=False)

education_income_df.head()

,year,region,scope,education_level,symbol,unit,value
20449,2019,Abrantes,Município,01. Inferior ao 1.º ciclo,NaN,euros,750.4
20450,2019,Abrantes,Município,02. Básico - 1º Ciclo,NaN,euros,939.7
20451,2019,Abrantes,Município,03. Básico - 2º Ciclo,NaN,euros,928.5
20452,2019,Abrantes,Município,04. Básico - 3º Ciclo,NaN,euros,933.5
20453,2019,Abrantes,Município,05. Secundário,NaN,euros,1027.3


## Population density

Data source: [PORDATA](https://www.pordata.pt/pt/estatisticas/populacao/populacao-residente/densidade-populacional)

In [24]:
df_density = pd.read_csv(DENSITY_RAW_FILE, encoding='utf-8-sig')
df_density.head()

,01. Ano,02. Nome País (Europa),03. Nome Região (Portugal),04. Âmbito Geográfico,05. Filtro 1,06. Filtro 2,07. Filtro 3,09. Símbolo,10. Valor,Max. 08. Escala
0,1990.0,Alemanha,NaN,NaN,NaN,NaN,NaN,NaN,222.2,pessoas por km2
1,1990.0,Áustria,NaN,NaN,NaN,NaN,NaN,NaN,93.0,pessoas por km2
2,1990.0,Bélgica,NaN,NaN,NaN,NaN,NaN,NaN,328.7,pessoas por km2
3,1990.0,Bulgária,NaN,NaN,NaN,NaN,NaN,NaN,77.4,pessoas por km2
4,1990.0,Chéquia,NaN,NaN,NaN,NaN,NaN,x,NaN,pessoas por km2


In [25]:
df_density['01. Ano'] = df_density['01. Ano'].astype('Int64')
df_density = df_density[df_density['04. Âmbito Geográfico'] == 'Município']
df_density.drop(columns=['02. Nome País (Europa)','04. Âmbito Geográfico','05. Filtro 1', '06. Filtro 2', '07. Filtro 3','09. Símbolo', "Max. 08. Escala "], inplace=True)

df_density = df_density[(df_density['01. Ano'] >= 2019) & (df_density['01. Ano'] <= 2023)]

df_density.rename(columns={"03. Nome Região (Portugal)": "municipality", "01. Ano": "year", "10. Valor": "people/km2"}, inplace=True)

df_density = df_density.reset_index(drop=True)

df_density.head()

,year,municipality,people/km2
0,2019,Abrantes,48.9
1,2019,Águeda,137.8
2,2019,Aguiar da Beira,26.0
3,2019,Alandroal,9.4
4,2019,Albergaria-a-Velha,156.4


In [26]:
df_density.to_csv(DENSITY_CLEAN_FILE, index=False)

## Age distribution

Dat source: [PORDATA](https://www.pordata.pt/pt/estatisticas/populacao/populacao-residente/populacao-residente-por-sexo-e-grupo-etario)

In [27]:
df_age = pd.read_csv(AGE_RAW_FILE, encoding='utf-8-sig')
df_age.head()

,01. Ano,02. Nome País (Europa),03. Nome Região (Portugal),04. Âmbito Geográfico,05. Filtro 1,06. Filtro 2,07. Filtro 3,08. Escala,09. Símbolo,10. Valor
0,1960.0,NaN,Portugal,NaN,Homens,5 - 9 anos,NaN,pessoas,NaN,433899.0
1,1960.0,NaN,Portugal,NaN,Homens,10 - 14 anos,NaN,pessoas,NaN,423614.0
2,1960.0,NaN,Portugal,NaN,Homens,15 - 19 anos,NaN,pessoas,NaN,366103.0
3,1960.0,NaN,Portugal,NaN,Homens,20 - 24 anos,NaN,pessoas,NaN,336672.0
4,1960.0,NaN,Portugal,NaN,Homens,25 - 29 anos,NaN,pessoas,NaN,324364.0


In [28]:
df_age['01. Ano'] = df_age['01. Ano'].astype('Int64')
df_age = df_age[df_age['04. Âmbito Geográfico'] == 'Município']
df_age = df_age[df_age["05. Filtro 1"] == "Total"]
df_age.drop(columns=["02. Nome País (Europa)","04. Âmbito Geográfico","05. Filtro 1", "07. Filtro 3","08. Escala ", "09. Símbolo"], inplace=True)
df_age = df_age[(df_age['01. Ano'] >= 2019) & (df_age['01. Ano'] <= 2023)]

df_age.rename(columns={"03. Nome Região (Portugal)": "municipality", "01. Ano": "year", "06. Filtro 2": "age_group", "10. Valor": "number_of_people"}, inplace=True)
df_age = df_age.reset_index(drop=True)

df_age = df_age[df_age['age_group'] != 'Total']
df_age['age_group'] = df_age['age_group'].str.replace(' anos', '').str.strip()

df_age = df_age.pivot(
    index = ['municipality', 'year'],
    columns="age_group",
    values="number_of_people",
).fillna(0)


In [29]:
df_age.to_csv(AGE_CLEAN_FILE)